In [1]:
#Random Forest with MinMax Scaler

In [2]:
import numpy as np 
import pandas as pd 

In [3]:
df=pd.read_csv("spam.csv",encoding="latin1")

In [4]:
#removing unnecessary coloums
df.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"],inplace=True)

In [5]:
df.rename(columns={"v1":"target","v2":"text"},inplace=True)

In [6]:
#encoding target
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [7]:
df["target"]=encoder.fit_transform(df["target"])

In [8]:
#checking for values
df.isnull().sum()

target    0
text      0
dtype: int64

In [9]:
#remove duplicates
df=df.drop_duplicates(keep="first")

df.duplicated().sum()

0

In [10]:
import nltk

In [11]:
from nltk.corpus import stopwords

In [12]:
import string

In [13]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [14]:
def transform_text(text):
    text=text.lower()

    text=nltk.word_tokenize(text)

    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text=y.copy()
    y.clear()
    for i in text:
        if i not in stopwords.words("english") and i not in string.punctuation:
            y.append(i)
    
    text=y.copy()
    y.clear()
    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [15]:
df["transformed_text"]=df["text"].apply(transform_text)
y=df["target"].values

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()

In [17]:
X=tfidf.fit_transform(df["transformed_text"]).toarray()

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [19]:
#using minmax scaler
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=scaler.fit_transform(X)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score
rfc=RandomForestClassifier(n_estimators=50,random_state=2)

In [27]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    accuracy=accuracy_score(y_test,y_pred)
    precision=precision_score(y_test,y_pred,average="binary")

    return accuracy,precision,recall

In [28]:
train_classifier(rfc,X_train,y_train,X_test,y_test)

NameError: name 'recall_score' is not defined

In [23]:
def check_spam(text, tfidf, scaler, model):
    transformed_text = transform_text(text)
    
    tfidf_vector = tfidf.transform([transformed_text]).toarray()
    
    scaled_vector = scaler.transform(tfidf_vector)
    
    prediction = model.predict(scaled_vector)
    
    return prediction[0]


def check(text_to_check):
    is_spam = check_spam(text_to_check, tfidf, scaler, rfc)
    if is_spam == 1:
        print("The text is spam.")
    else:
        print("The text is not spam.")


In [24]:
txt=input("Enter your msg: ")
print(txt)
check(txt)

Hi I am you
The text is not spam.
